In [1]:
# Import necessary libraries 
import requests
import json
import pandas as pd
import time

In [2]:
# There is no API interface to get the data for all states at the same time. There is an API interface to get the data 
# by specific state. I determined that it might be better to get the data for all states one at a time and combine the 
# data. There is an interface to get a list of all valid state codes, that can be used to retrieve data by states.

In [3]:
# Open the APIkeys.json file and extract api key
with open('APIkeys.json') as f:
    keys = json.load(f)
    aqsapi = keys['AQSapi']

In [4]:
# Setup a variable for the url to get the list of all states 
states_url = 'https://aqs.epa.gov/data/api/list/states?email=akolekar@my365.bellevue.edu&key='+aqsapi

In [5]:
# Define a function to get api data for the url passed in as a parameter
def get_api_data(url):
    # Use requests library to get data using the url variable
    response = requests.get(url)
        
    # Load the url data in json format
    json_data=json.loads(response.text)
    return json_data

In [6]:
# Call get_api_data function for states_url and store the json data in the json_data variable 
json_data=get_api_data(states_url)

In [7]:
# Create an empty list to store state codes
states = []

In [8]:
# Loop through the json data and get the state code value for each instance. Store the data in the states list.
for i in range(len(json_data['Data'])):
    states.append(json_data['Data'][i]['code'])

In [9]:
# print the state codes and compare with the website to make sure the values are correct.
print(states)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '66', '72', '78', '80', 'CC']


In [10]:
# Check the number of state codes and validate the count by comparing with the website. 
print(len(states))

56


In [11]:
# Due to performance issues, spilt the states list in three lists
states1 = states[0:20]
states2 = states[20:40]
states3 = states[40:56]

In [12]:
# Confirm that the states are properly split 
print(states1)
print(states2)
print(states3)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23']
['24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44']
['45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '66', '72', '78', '80', 'CC']


In [13]:
# Setup headers list with the keys from the json data
headers = ['address', 'cbsa_code', 'cbsa_name', 'city_name', 'close_date', 'concurred_exclusions', 'county_code',
           'county_name', 'csa_code', 'csa_name', 'datum', 'dominant_source', 'elevation', 'last_method_begin_date',
           'last_method_code', 'last_method_description', 'lat_lon_accuracy', 'latitude', 'local_site_name',
           'longitude', 'measurement_scale', 'measurement_scale_def', 'monitor_type', 
           'monitoring_agency', 'monitoring_agency_code', 'monitoring_objective', 'naaqs_primary_monitor', 
           'networks', 'open_date', 'parameter_code', 'parameter_name', 'pl_probe_location', 'poc', 'probe_height', 
           'qa_primary_monitor', 'si_id', 'site_number', 'state_code', 'state_name', 'tribal_code', 'tribe_name']


In [14]:
# Define a function to process json data and extract values for each key for each county. The function should then 
# combine the processed data and headers list into a data frame and return the data frame. 

def process_json_data(json_data, headers):
    county_list = []                                       # Empty list to store data of all counties of the state
    for i in range(len(json_data['Data'])):                # Loop through all instances of 'Data' key 
        county_data = []                                   # Empty list to store data of a specific county data
        for k in headers:                                  # Loop through all values in the headers list
            county_data.append(json_data['Data'][0][k])    # Extract value of each header and add to the county_data list
        county_list.append(county_data)                    # Add the county_data to the list of all counties
        
    # Print number of counties for each state if there are counties
    if len(county_list) > 0:
        print("There are {} counties in {}.".format(len(county_list), county_list[0][-3]))
    
    # Use the rows from the county_list and the headers list to create a data frame state_df
    state_df = pd.DataFrame(county_list, columns=headers)
    
    # Return the data frame
    return state_df

In [15]:
# Create an empty data frame using headers list as columns
combined_df = pd.DataFrame(columns=headers)

# Setup the variables for url
serviceurl1 = 'https://aqs.epa.gov/data/api/monitors/byState?email=akolekar@my365.bellevue.edu&key='
serviceurl2 = '&bdate=20100101&edate=20160101&state='

# Loop through the states1 list
for i in range(len(states1)):
    # Process the data only if the state value is numeric. The API will fail for non-numeric values. 
    if states1[i].isnumeric():
        # Setup the url variable using key, state code, and other variables
        url = serviceurl1+aqsapi+serviceurl2+states1[i]
        
        # Call get_api_data function using url to get the json data
        json_data = get_api_data(url)
        
        # Create the frames list using combined_df and the df returned by process_json_data function
        frames = [combined_df, process_json_data(json_data, headers)]
        
        # The state level data gets added to combined_df after every state is processed. Merge the dfs using concat method.
        combined_df = pd.concat(frames)
        
        # Add a delay of 1 second between API calls to prevent connection issues  
        time.sleep(1)


There are 1562 counties in Alabama.
There are 833 counties in Alaska.
There are 2776 counties in Arizona.
There are 425 counties in Arkansas.
There are 12366 counties in California.
There are 3726 counties in Colorado.
There are 735 counties in Connecticut.
There are 369 counties in Delaware.
There are 542 counties in District Of Columbia.
There are 2699 counties in Florida.
There are 2047 counties in Georgia.
There are 332 counties in Hawaii.
There are 644 counties in Idaho.
There are 2570 counties in Illinois.
There are 2711 counties in Indiana.
There are 1658 counties in Iowa.
There are 482 counties in Kansas.
There are 2218 counties in Kentucky.
There are 934 counties in Louisiana.
There are 1004 counties in Maine.


In [18]:
# Check the number of counties after states1 list is processed
combined_df.shape

(40633, 41)

In [19]:
# Setup the variables for url
serviceurl1 = 'https://aqs.epa.gov/data/api/monitors/byState?email=akolekar@my365.bellevue.edu&key='
serviceurl2 = '&bdate=20100101&edate=20160101&state='

# Loop through the states2 list
for i in range(len(states2)):
    # Process the data only if the state value is numeric. The API will fail for non-numeric values. 
    if states2[i].isnumeric():
        # Setup the url variable using key, state code, and other variables
        url = serviceurl1+aqsapi+serviceurl2+states2[i]

        # Call get_api_data function using url to get the json data
        json_data = get_api_data(url)

        # Create the frames list using combined_df and the df returned by process_json_data function
        frames = [combined_df, process_json_data(json_data, headers)]

        # The state level data gets added to combined_df after every state is processed. Merge the dfs using concat method.
        combined_df = pd.concat(frames)
        
        # Add a delay of 1 second between API calls to prevent connection issues  
        time.sleep(1)


There are 1013 counties in Maryland.
There are 1878 counties in Massachusetts.
There are 2504 counties in Michigan.
There are 3957 counties in Minnesota.
There are 1167 counties in Mississippi.
There are 1308 counties in Missouri.
There are 898 counties in Montana.
There are 339 counties in Nebraska.
There are 667 counties in Nevada.
There are 765 counties in New Hampshire.
There are 1414 counties in New Jersey.
There are 943 counties in New Mexico.
There are 3244 counties in New York.
There are 2534 counties in North Carolina.
There are 475 counties in North Dakota.
There are 3942 counties in Ohio.
There are 2055 counties in Oklahoma.
There are 2501 counties in Oregon.
There are 5029 counties in Pennsylvania.
There are 793 counties in Rhode Island.


In [20]:
# Check the number of counties after states1 and states2 lists are processed
combined_df.shape

(78059, 41)

In [21]:
# Setup the variables for url
serviceurl1 = 'https://aqs.epa.gov/data/api/monitors/byState?email=akolekar@my365.bellevue.edu&key='
serviceurl2 = '&bdate=20100101&edate=20160101&state='

# Loop through the states3 list
for i in range(len(states3)):
    # Process the data only if the state value is numeric. The API will fail for non-numeric values. 
    if states3[i].isnumeric():
        # Setup the url variable using key, state code, and other variables
        url = serviceurl1+aqsapi+serviceurl2+states3[i]

        # Call get_api_data function using url to get the json data
        json_data = get_api_data(url)

        # Create the frames list using combined_df and the df returned by process_json_data function
        frames = [combined_df, process_json_data(json_data, headers)]

        # The state level data gets added to combined_df after every state is processed. Merge the dfs using concat method.
        combined_df = pd.concat(frames)
        
        # Add a delay of 1 second between API calls to prevent connection issues  
        time.sleep(1)


There are 1162 counties in South Carolina.
There are 632 counties in South Dakota.
There are 1346 counties in Tennessee.
There are 8653 counties in Texas.
There are 1310 counties in Utah.
There are 730 counties in Vermont.
There are 1498 counties in Virginia.
There are 1826 counties in Washington.
There are 758 counties in West Virginia.
There are 1580 counties in Wisconsin.
There are 1868 counties in Wyoming.
There are 186 counties in Puerto Rico.
There are 253 counties in Virgin Islands.
There are 315 counties in Country Of Mexico.


In [22]:
# Check the number of counties after all states are processed
combined_df.shape

(100176, 41)

In [23]:
# Check first few records in the dataframe using head method.
combined_df.head()

,address,cbsa_code,cbsa_name,city_name,close_date,concurred_exclusions,county_code,county_name,csa_code,csa_name,...,pl_probe_location,poc,probe_height,qa_primary_monitor,si_id,site_number,state_code,state_name,tribal_code,tribe_name
0,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",19300,"Daphne-Fairhope-Foley, AL",Fairhope,2010-09-17,None,003,Baldwin,380,"Mobile-Daphne-Fairhope, AL",...,None,10,None,Y,7,0010,01,Alabama,None,None
1,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",19300,"Daphne-Fairhope-Foley, AL",Fairhope,2010-09-17,None,003,Baldwin,380,"Mobile-Daphne-Fairhope, AL",...,None,10,None,Y,7,0010,01,Alabama,None,None
2,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",19300,"Daphne-Fairhope-Foley, AL",Fairhope,2010-09-17,None,003,Baldwin,380,"Mobile-Daphne-Fairhope, AL",...,None,10,None,Y,7,0010,01,Alabama,None,None
3,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",19300,"Daphne-Fairhope-Foley, AL",Fairhope,2010-09-17,None,003,Baldwin,380,"Mobile-Daphne-Fairhope, AL",...,None,10,None,Y,7,0010,01,Alabama,None,None
4,"FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",19300,"Daphne-Fairhope-Foley, AL",Fairhope,2010-09-17,None,003,Baldwin,380,"Mobile-Daphne-Fairhope, AL",...,None,10,None,Y,7,0010,01,Alabama,None,None


In [24]:
# Review the columns names to see if any column headings need to be changed

combined_df.columns

Index(['address', 'cbsa_code', 'cbsa_name', 'city_name', 'close_date',
       'concurred_exclusions', 'county_code', 'county_name', 'csa_code',
       'csa_name', 'datum', 'dominant_source', 'elevation',
       'last_method_begin_date', 'last_method_code', 'last_method_description',
       'lat_lon_accuracy', 'latitude', 'local_site_name', 'longitude',
       'measurement_scale', 'measurement_scale_def', 'monitor_type',
       'monitoring_agency', 'monitoring_agency_code', 'monitoring_objective',
       'naaqs_primary_monitor', 'networks', 'open_date', 'parameter_code',
       'parameter_name', 'pl_probe_location', 'poc', 'probe_height',
       'qa_primary_monitor', 'si_id', 'site_number', 'state_code',
       'state_name', 'tribal_code', 'tribe_name'],
      dtype='object')

In [25]:
# Rename column names for City, County, and State

combined_df.rename(columns={'city_name': 'City', 
                            'county_name': 'County',
                            'state_name':'State'}, inplace=True)

In [26]:
# Confirm that the column names are updated

combined_df.columns

Index(['address', 'cbsa_code', 'cbsa_name', 'City', 'close_date',
       'concurred_exclusions', 'county_code', 'County', 'csa_code', 'csa_name',
       'datum', 'dominant_source', 'elevation', 'last_method_begin_date',
       'last_method_code', 'last_method_description', 'lat_lon_accuracy',
       'latitude', 'local_site_name', 'longitude', 'measurement_scale',
       'measurement_scale_def', 'monitor_type', 'monitoring_agency',
       'monitoring_agency_code', 'monitoring_objective',
       'naaqs_primary_monitor', 'networks', 'open_date', 'parameter_code',
       'parameter_name', 'pl_probe_location', 'poc', 'probe_height',
       'qa_primary_monitor', 'si_id', 'site_number', 'state_code', 'State',
       'tribal_code', 'tribe_name'],
      dtype='object')

In [27]:
# The missing values are identified as null and can be checked by using isnull() function. Find missing values for each column
# and add them by using sum() function to determine how many missing values exist for each column.

combined_df.isnull().sum()

address                         0
cbsa_code                    3686
cbsa_name                    3686
City                            0
close_date                  42399
concurred_exclusions       100176
county_code                     0
County                          0
csa_code                    22629
csa_name                    22629
datum                           0
dominant_source             66559
elevation                       0
last_method_begin_date          0
last_method_code                0
last_method_description         0
lat_lon_accuracy                0
latitude                        0
local_site_name              7611
longitude                       0
measurement_scale           48663
measurement_scale_def       48663
monitor_type                44055
monitoring_agency               0
monitoring_agency_code          0
monitoring_objective            0
naaqs_primary_monitor       98399
networks                    65519
open_date                       0
parameter_code

In [28]:
# Remove columns concurred_exclusions, dominant_source, naaqs_primary_monitor, networks, probe_height, 
# qa_primary_monitor, tribal_code, and tribe_name as more than half of the values in these columns are null

del combined_df['concurred_exclusions']
del combined_df['dominant_source']
del combined_df['naaqs_primary_monitor']
del combined_df['networks']
del combined_df['probe_height']
del combined_df['qa_primary_monitor']
del combined_df['tribal_code']
del combined_df['tribe_name']

In [29]:
# Confirm that the column names are deleted

combined_df.columns

Index(['address', 'cbsa_code', 'cbsa_name', 'City', 'close_date',
       'county_code', 'County', 'csa_code', 'csa_name', 'datum', 'elevation',
       'last_method_begin_date', 'last_method_code', 'last_method_description',
       'lat_lon_accuracy', 'latitude', 'local_site_name', 'longitude',
       'measurement_scale', 'measurement_scale_def', 'monitor_type',
       'monitoring_agency', 'monitoring_agency_code', 'monitoring_objective',
       'open_date', 'parameter_code', 'parameter_name', 'pl_probe_location',
       'poc', 'si_id', 'site_number', 'state_code', 'State'],
      dtype='object')

In [30]:
# While there are other columns with some null values, it is important to not delete any rows since the rows are unique to
# the combination of county and state. The columns, such as State, County, and City do not have any null values.

In [31]:
# Check for duplicate values in 'County' and 'State' columns

print("City is duplicated - {}".format(any(combined_df.City.duplicated())))
print("State is duplicated - {}".format(any(combined_df.State.duplicated())))
print("County is duplicated - {}".format(any(combined_df.County.duplicated())))

City is duplicated - True
State is duplicated - True
County is duplicated - True


In [32]:
# Since the 'State' value was duplicated for each corresponding county, it is obvious to have duplicate values for state.
# It is also possible to have same county name or same city name in multiple states. 
# Since the duplicates are allowed, no additional action is needed to handle duplicate values.

In [33]:
# Print the first row in the data frame to understand the values in each column
print(combined_df.iloc[[0]])

                                             address cbsa_code  \
0  FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...     19300   

                   cbsa_name      City  close_date county_code   County  \
0  Daphne-Fairhope-Foley, AL  Fairhope  2010-09-17         003  Baldwin   

  csa_code                    csa_name  datum  ...  monitoring_objective  \
0      380  Mobile-Daphne-Fairhope, AL  NAD83  ...   POPULATION EXPOSURE   

    open_date parameter_code    parameter_name  pl_probe_location  poc si_id  \
0  2010-05-17          43236  2-Ethyl-1-butene               None   10     7   

   site_number state_code    State  
0         0010         01  Alabama  

[1 rows x 33 columns]


In [34]:
# Most of the columns have consistent values. State, County, and City values are consistent and are stored with title case
# (first character of each word to uppercase and remaining to lowercase). The Address column values need to be converted 
# to title case. 

combined_df['address'] = combined_df['address'].str.title()

# Check if the changes are applied using head() function
combined_df.head()

,address,cbsa_code,cbsa_name,City,close_date,county_code,County,csa_code,csa_name,datum,...,monitoring_objective,open_date,parameter_code,parameter_name,pl_probe_location,poc,si_id,site_number,state_code,State
0,"Fairhope High School, 1 Pirate Drive, Fairhope...",19300,"Daphne-Fairhope-Foley, AL",Fairhope,2010-09-17,003,Baldwin,380,"Mobile-Daphne-Fairhope, AL",NAD83,...,POPULATION EXPOSURE,2010-05-17,43236,2-Ethyl-1-butene,None,10,7,0010,01,Alabama
1,"Fairhope High School, 1 Pirate Drive, Fairhope...",19300,"Daphne-Fairhope-Foley, AL",Fairhope,2010-09-17,003,Baldwin,380,"Mobile-Daphne-Fairhope, AL",NAD83,...,POPULATION EXPOSURE,2010-05-17,43236,2-Ethyl-1-butene,None,10,7,0010,01,Alabama
2,"Fairhope High School, 1 Pirate Drive, Fairhope...",19300,"Daphne-Fairhope-Foley, AL",Fairhope,2010-09-17,003,Baldwin,380,"Mobile-Daphne-Fairhope, AL",NAD83,...,POPULATION EXPOSURE,2010-05-17,43236,2-Ethyl-1-butene,None,10,7,0010,01,Alabama
3,"Fairhope High School, 1 Pirate Drive, Fairhope...",19300,"Daphne-Fairhope-Foley, AL",Fairhope,2010-09-17,003,Baldwin,380,"Mobile-Daphne-Fairhope, AL",NAD83,...,POPULATION EXPOSURE,2010-05-17,43236,2-Ethyl-1-butene,None,10,7,0010,01,Alabama
4,"Fairhope High School, 1 Pirate Drive, Fairhope...",19300,"Daphne-Fairhope-Foley, AL",Fairhope,2010-09-17,003,Baldwin,380,"Mobile-Daphne-Fairhope, AL",NAD83,...,POPULATION EXPOSURE,2010-05-17,43236,2-Ethyl-1-butene,None,10,7,0010,01,Alabama


In [35]:
# Since the data is specific to counties and is using values from specific categories for each column, there are no outliers.

In [36]:
combined_df.to_csv('aqs.csv', index=False)